# Guppies data processing with Ecodylic Behavior


Data, to calculate:

Index(['Trial', 'FishID', , 'time', 'frame', 'ANGLE', 'x',
       'y', 'focal', 

FFGROUPS:
'group_size', 'event', 'group_membership'

1) ffgroups-metrics
    CALCULATE - AFTER FFGROUPS - PER FRAME
    'distance_from_centroid', 'xrot_to_centroid', 'yrot_to_centroid', 'dev_speed_to_mean'
    CALCULATE - AFTER FFGROUPS - SUMMARY
    'fractime_norm2', 'avg_duration_frame', 'med_duration_frame', 'ftime_periphery', 'ftime_periphery_norm', 


2) speed-angvel
CALCULATE - SIMPLE:
speed, 'angularspeed' 'speed-std', 'RMS_delta_angle' (maybe not needed - angular speed is used)

3) nearestneighbor
CALCULATE - DIST.  saves:  nn_id, nn_delta_x, nn_delta_y, nn_dist, nn_(diff in ego centric coords?)
'nndist', 'nn_dist-std'


?? or not needed:
'frame_id_count', 'frame_gsize_sum', 'frame_id_sum',
'norm_ftime2'
'ftime_front_norm',  'ftime_front'  
       'fractime_norm2_trialnorm'
       'speed_trialnormall'

4) social forces
   ** NEEDS nndist  (feature requires inputset, with tracks and nearest)



In [1]:
import pandas as pd
filename = '/Volumes/JD-SSD/guppies/guppies/dataset/tracks/control_treatment__t59_d5.parquet'
pd.read_parquet(filename)

,frame,time,id,group,sequence,X#wcentroid,Y#wcentroid,ANGLE,SPEED#wcentroid,ACCELERATION#wcentroid,border_distance_cm,midline_length,x,y,Focal_fish,bad_frame
0,108001,3599.909424,0,control_treatment,t59_d5,8.703287,23.131819,-2.570283,1.326016,45.204636,8.736189,17.571571,8.622351,22.916702,0,0
1,108001,3599.909424,1,control_treatment,t59_d5,8.554715,22.607578,2.527856,2.219128,17.820446,8.586458,11.594145,8.475160,22.397339,0,0
2,108001,3599.909424,2,control_treatment,t59_d5,14.362971,13.778959,2.239858,2.227515,17.878916,13.676469,40.618824,14.229402,13.650820,0,0
3,108001,3599.909424,3,control_treatment,t59_d5,7.470642,27.537029,0.340293,0.490705,19.561220,7.314008,46.053547,7.401168,27.280945,0,0
4,108001,3599.909424,4,control_treatment,t59_d5,8.175490,20.321123,-0.215024,0.971326,11.904131,8.020432,44.742889,8.099462,20.132145,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828613,339985,11332.468750,3,control_treatment,t59_d5,63.349174,9.468217,2.828627,1.011021,4.859296,9.431363,45.866978,62.760052,9.380167,0,0
1828614,339985,11332.468750,4,control_treatment,t59_d5,65.453857,7.264638,3.132723,1.425061,12.423551,7.257699,47.866180,64.845161,7.197080,0,0
1828615,339985,11332.468750,5,control_treatment,t59_d5,NaN,NaN,NaN,NaN,NaN,5.609266,NaN,70.451675,5.564590,0,0
1828616,339985,11332.468750,6,control_treatment,t59_d5,68.578171,4.781428,2.396162,0.523825,23.926695,4.779736,26.149790,67.940422,4.736963,0,0


In [3]:
import pandas as pd
filename = '/Volumes/JD-SSD/guppies/guppies/dataframes/guppy_8_t21_d1_processing3.pkl'
df = pd.read_pickle(filename)
df.columns

Index(['FishID', 'event', 'time', 'Trial', 'frame', 'X#wcentroid',
       'Y#wcentroid', 'ANGLE', 'border_distance_cm', 'SPEED#wcentroid',
       'ACCELERATION#wcentroid', 'midline_length', 'x', 'y', 'Focal_fish',
       'trex_tracking_error', 'time_diff', 'x_diff', 'y_diff', 'speed',
       'acceleration', 'heading', 'dir_x', 'dir_y', 'dirx_angle', 'diry_angle',
       'midlen', 'BL', 'dt', 'skipped_frames', 'speed_original',
       'filtered_speed_cat', 'bad_frame', 'all_errors', 'group_membership',
       'group_size', 'delta_x', 'delta_y', 'step_distance',
       'cumulative_distance', 'turning_rate', 'turning_rate_abs', 'centroid_x',
       'centroid_y', 'centroid_heading', 'centroid_dirx', 'centroid_diry',
       'centroid_speed', 'polarization_group', 'polarization_group_trans',
       'distance_from_centroid', 'angle_from_centroid', 'xrot_to_centroid',
       'yrot_to_centroid', 'norm_speed_to_median', 'norm_speed_to_mean',
       'dev_speed_to_mean', 'dev_speed_to_median', 'de

In [6]:
df['group_membership'].unique()

array([2., 0., 3., 1., 4., 5., 6., 7.])

# Load dataset and convert to standard format

In [9]:
import importlib
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
from behavior import dataset as ds
import behavior.features as features
import behavior.analysis as analysis

importlib.reload(ds)
importlib.reload(features)
importlib.reload(analysis)


<module 'behavior.analysis' from '/Users/jacob/Documents/GitHub/EcodylicScience/behavior/src/behavior/analysis.py'>

## Load

In [10]:
import re
import json
from pathlib import Path

import pandas as pd
import yaml
from behavior.helpers import to_safe_name

# ---------------------------------------------------------------------
# 1) Define dataset config for "guppies"
# ---------------------------------------------------------------------
DATASET_CONFIGS = {
    "guppies": {
        "manifest": "/Volumes/JD-SSD/guppies/guppies/dataset/dataset.yaml",
        "manifest_params": {
            "name": "guppies",
            "base_dir": "/Volumes/JD-SSD/guppies/guppies/dataset/",
            "version": "0.1.0",
            "index_format": "group/sequence",
        },
        # optional metadata used by your own tooling
        "groups": None,
        "label_kind": None,
        "media_index": None,
        "tracks_index": {
            "search_dirs": [
                "/Volumes/JD-SSD/guppies/guppies/dataframes",
                "/Volumes/JD-SSD/guppies/guppies/dataframes_control",
            ],
            "patterns": "*processing3.pkl",
            # here we are doing a one-off manual conversion, so src_format
            # is just informative; the code below doesn't use it.
            "src_format": "guppies_pkl",
            "multi_sequences_per_file": False,
        },
        "convert_tracks": {
            "merge_per_sequence": False,
        },
    },
}

ACTIVE_DATASET = "guppies"
CFG = DATASET_CONFIGS[ACTIVE_DATASET]
print(f"Using dataset '{ACTIVE_DATASET}'")


Using dataset 'guppies'


In [11]:
manifest_path = Path(CFG["manifest"])
if not manifest_path.exists():
    manifest_args = CFG.get("manifest_params")
    if not manifest_args:
        raise FileNotFoundError(f"Manifest {manifest_path} is missing and no creation params were provided.")
    created = ds.new_dataset_manifest(**manifest_args)
    print(f"Created manifest at {created}")

dataset = ds.Dataset(str(manifest_path))
dataset.load()
print(f"Loaded dataset '{dataset.name}' -> {manifest_path}")


Loaded dataset 'guppies' -> /Volumes/JD-SSD/guppies/guppies/dataset/dataset.yaml


## Convert tracks

In [4]:


# ---------------------------------------------------------------------
# 3) Helpers for conversion
# ---------------------------------------------------------------------
def get_sequence(src_path: Path) -> str:
    """
    Extract 't21_d1' from 'guppy_8_t21_d1_processing3.pkl'.
    Adjust the regex if your naming differs.
    """
    stem = src_path.stem  # e.g., "guppy_8_t21_d1_processing3"
    m = re.search(r"_t(\d+_d\d+)", stem)
    if not m:
        # fallback: whole stem
        return stem
    return f"t{m.group(1)}"  # "t21_d1"


def iter_group_files():
    """
    Yield (group_name, list_of_files) for each experimental group.
    Adjust group names as you like.
    """

    root = Path("/Volumes/JD-SSD/guppies/guppies/dataframes")
    pattern = re.compile(r"_d1(_|$)")
    
    files_exp_baseline = sorted(
        p for p in root.glob("*_processing3.pkl")
        if pattern.search(p.stem)
    )
    files_exp_treatment = sorted(
        p for p in root.glob("*_processing3.pkl")
        if not pattern.search(p.stem)
    )
    
    root = Path("/Volumes/JD-SSD/guppies/guppies/dataframes_control/")
    files_control_baseline = sorted(
        p for p in root.glob("*_processing3.pkl")
        if pattern.search(p.stem)
    )
    files_control_treatment = sorted(
        p for p in root.glob("*_processing3.pkl")
        if not pattern.search(p.stem)
    )    

    yield "exp_baseline", files_exp_baseline
    yield "exp_treatment", files_exp_treatment
    yield "control_baseline", files_control_baseline
    yield "control_treatment", files_control_treatment

# ---------------------------------------------------------------------
# 4) Convert PKL -> tracks parquet + save per-ID focal labels
# ---------------------------------------------------------------------
tracks_root = dataset.get_root("tracks")
tracks_root.mkdir(parents=True, exist_ok=True)
print("Tracks root:", tracks_root)

for group, file_list in iter_group_files():
    print(f"\n=== Converting group '{group}' ({len(file_list)} files) ===")
    for src in file_list:
        src = Path(src)
        sequence = get_sequence(src)

        out_path = tracks_root / f"{group}__{sequence}.parquet"

        # ---- load source PKL ---------------------------------------------------
        df = pd.read_pickle(src)

        # ---- normalize to standard tracks schema ------------------------------
        trex_df = pd.DataFrame({
            "frame": df["frame"].astype("int32"),
            "time": df["time"].astype("float32", copy=False),
            "id": df["FishID"].astype("int32"),
            "group": group,
            "sequence": sequence,
            # main kinematic fields (rename to your canonical names if needed)
            "X#wcentroid": df["X#wcentroid"].astype("float32", copy=False),
            "Y#wcentroid": df["Y#wcentroid"].astype("float32", copy=False),
            "ANGLE": df["ANGLE"].astype("float32", copy=False),
            "SPEED#wcentroid": df["SPEED#wcentroid"].astype("float32", copy=False),
            "ACCELERATION#wcentroid": df["ACCELERATION#wcentroid"].astype("float32", copy=False),
            # extra numeric features you might want:
            "border_distance_cm": df["border_distance_cm"].astype("float32", copy=False),
            "midline_length": df["midline_length"].astype("float32", copy=False),
            "x": df["x"].astype("float32", copy=False),
            "y": df["y"].astype("float32", copy=False),
            # flags / errors / focal info:
            "Focal_fish": df["Focal_fish"].astype("int8"),
            "bad_frame": df["bad_frame"].astype("int8"),
        })

        trex_df = trex_df.sort_values(["frame", "id"]).reset_index(drop=True)

        out_path.parent.mkdir(parents=True, exist_ok=True)
        trex_df.to_parquet(out_path, index=False)
        print("Saved tracks ->", out_path)

        # ---- per-ID focal labels (id_tags) ------------------------------------
        focal_per_id = (
            df.groupby("FishID")["Focal_fish"]
              .max()      # assume 0/1 flag; if not, adjust
              .astype(bool)
              .to_dict()
        )

        per_id_labels = {
            int(fid): {"focal": bool(is_focal)}
            for fid, is_focal in focal_per_id.items()
        }

        # This assumes you have dataset.save_id_labels implemented.
        dataset.save_id_labels(
            kind="id_tags",
            group=group,
            sequence=sequence,
            per_id_labels=per_id_labels,
            metadata={"source_file": src.name},
            overwrite=True,
        )
        print(f"Saved focal tags for sequence '{sequence}'")

# ---------------------------------------------------------------------
# 5) Build tracks/index.csv so Dataset can discover the sequences
# ---------------------------------------------------------------------
index_rows = []
for p in sorted(tracks_root.glob("*.parquet")):
    stem = p.stem  # "group__sequence"
    if "__" in stem:
        group, sequence = stem.split("__", 1)
    else:
        group, sequence = "", stem

    # read minimal info to populate index
    df = pd.read_parquet(p, columns=["frame"])
    n_frames = len(df)
    first_frame = int(df["frame"].min())
    last_frame = int(df["frame"].max())

    index_rows.append({
        "group": group,
        "sequence": sequence,
        "sequence_safe": to_safe_name(sequence),
        "abs_path": str(p),
        "n_frames": n_frames,
        "first_frame": first_frame,
        "last_frame": last_frame,
    })

idx_df = pd.DataFrame(index_rows)
idx_path = tracks_root / "index.csv"
idx_df.to_csv(idx_path, index=False)
print("\nWrote tracks index ->", idx_path)
print("Indexed", len(idx_df), "tracks files.")

Tracks root: /Volumes/JD-SSD/guppies/guppies/dataset/tracks

=== Converting group 'exp_baseline' (24 files) ===
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t21_d1.parquet
Saved focal tags for sequence 't21_d1'
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t22_d1.parquet
Saved focal tags for sequence 't22_d1'
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t23_d1.parquet
Saved focal tags for sequence 't23_d1'
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t24_d1.parquet
Saved focal tags for sequence 't24_d1'
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t25_d1.parquet
Saved focal tags for sequence 't25_d1'
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t27_d1.parquet
Saved focal tags for sequence 't27_d1'
Saved tracks -> /Volumes/JD-SSD/guppies/guppies/dataset/tracks/exp_baseline__t36_d1.parquet
Saved foca

# Feature calculation

In [53]:
%%time
import importlib, feature_library.ffgroups as ffgroups
importlib.reload(ffgroups)

# 1) Run ffgroups on a subset of sequences (from tracks)
run_id = dataset.run_feature(
    ffgroups.FFGroups({"distance_cutoff": 20.32, "window_size": 5, "min_event_duration": 1}),
    input_kind="tracks",
    # groups=["exp_baseline"],    # or None for all
    # sequences = dataset.list_sequences(group="exp_baseline"),
    parallel_workers = 3,
    parallel_mode = "process"
)

Process SpawnProcess-8:
Process SpawnProcess-7:
Process SpawnProcess-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/jacob/miniforge3/envs/beesbook/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/jacob/miniforge3/envs/beesbook/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jacob/miniforge3/envs/beesbook/lib/python3.12/concurrent/futures/process.py", line 252, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacob/miniforge3/envs/beesbook/lib/python3.12/multiprocessing/queues.py", line 102, in get
    with self._rlock:
         ^^^^^^^^^^^
  File "/Users/jacob/miniforge3/envs/beesbook/lib/python3.12/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
  File "/Us

In [18]:
# 2) Create an inputset that holds tracks + ffgroups outputs
# run_id = "0.1-e379e8cd17"
ds.save_inputset(
    dataset,
    name="tracks+ffgroups",
    inputs=[
        {"kind": "tracks"},  # raw ids/metadata  "columns": ["frame", "time", "id", "group", "sequence"]
        {"feature": "ffgroups", "run_id": run_id},                                   # this ffgroups run
    ],
    overwrite=True,
)

PosixPath('/Volumes/JD-SSD/guppies/guppies/dataset/inputsets/tracks%2Bffgroups.json')

In [19]:
# 3) Load one sequence into a DataFrame for inspection
# Pick the latest run of the inputset’s feature entries; tracks stream directly
target_sequence = 't21_d1'
dfs = []
for g, s, df in ds._yield_inputset_frames(
    dataset,  # internal helper; fine for inspection
    inputset_name="tracks+ffgroups",
    sequences=[target_sequence],
):
    # df here is the tracks table; join with ffgroups parquet for the same seq
    dfs.append(df)

# Load the ffgroups output parquet for the same sequence/run
safe_seq = to_safe_name(target_sequence)

ff_root = dataset.get_root("features") / f"ffgroups/{run_id}/"
ff_paths = sorted(ff_root.glob(f"*__{safe_seq}.parquet"))  # matches any group__<seq>.parquet
ff_df = pd.read_parquet(ff_paths[0]) if ff_paths else pd.DataFrame()

tracks_df = pd.concat(dfs, ignore_index=True)
merged = tracks_df.merge(ff_df, how="left", on=["frame", "id", "group", "sequence"])
display(merged.head())

,frame,time_x,id,group,sequence,X#wcentroid,Y#wcentroid,ANGLE,SPEED#wcentroid,ACCELERATION#wcentroid,border_distance_cm,midline_length,x,y,Focal_fish,bad_frame,group_membership,group_size,time_y,event
0,108001,3602.594238,0,exp_baseline,t21_d1,69.203613,88.964195,-0.005224,0.234207,14.880315,31.029045,29.967735,68.560051,88.136864,0,0,0.0,2,3602.594238,0
1,108001,3602.594238,1,exp_baseline,t21_d1,111.893616,110.061966,1.114550,2.747268,37.429714,8.098183,31.673729,110.853043,109.038429,1,0,1.0,2,3602.594238,1
2,108001,3602.594238,2,exp_baseline,t21_d1,56.710899,72.049995,0.504916,2.935105,14.312032,47.984619,23.183081,56.183510,71.379959,0,0,2.0,1,3602.594238,27
3,108001,3602.594238,3,exp_baseline,t21_d1,106.361130,33.800343,0.102539,1.299904,15.424468,13.788536,32.029694,105.372017,33.486012,0,0,3.0,2,3602.594238,6
4,108001,3602.594238,4,exp_baseline,t21_d1,101.645119,35.509937,0.533460,5.504618,19.183117,18.561928,55.136238,100.699860,35.179710,0,0,3.0,2,3602.594238,6
